## Import

In [1]:
import pandas as pd

## Code

In [3]:
df = pd.read_csv('../../dataset/Appenzeller-Herzog_2020.csv')
print(df.head())

   record_id                                              title  \
0          1  Binding Selectivity of Methanobactin from Meth...   
1          2  Characteristics and prevalence of Wilson's dis...   
2          3  Modality of treatment and potential outcome of...   
3          4  The early history of manganese and the recogni...   
4          5  Zinc monotherapy for young children with presy...   

                                            abstract keywords  \
0  Methanobactin (Mb) from Methylosinus trichospo...      NaN   
1  BACKGROUND AND AIMS: Only a few epidemiologica...      NaN   
2  BACKGROUND/PURPOSE: This study aimed to invest...      NaN   
3  The history of the biomedical recognition mang...      NaN   
4  BACKGROUND AND AIM: Few studies of zinc monoth...      NaN   

                                             authors  year    date  \
0             McCabe, J. W.;Vangala, R.;Angel, L. A.  2017  Aug 30   
1  Poujois, A.;Woimant, F.;Samson, S.;Chaine, P.;...  2017  Jun 22